In [1]:
# from dotenv import load_dotenv
# from langchain_groq import ChatGroq

from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

In [2]:
# llm=ChatGroq(model="deepseek-r1-distill-llama-70b")
# llm.invoke("hi")

llm = ChatOllama(model='llama3.2:latest')

In [3]:
# With bind.
chain = (
    llm.bind(stop=["three"])
    | StrOutputParser()
)

chain.invoke("Repeat quoted words exactly: 'One two three four five.'")

'One two '

In [14]:
# Without bind.
chain = (
    llm
    | StrOutputParser()
)

chain.invoke("Repeat quoted words exactly: 'One two three four five.'")

'"One two three four five."'

In [36]:
from langchain.tools import tool
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field
from typing import ClassVar, Type

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """
    return a * b

In [ ]:
multiply

StructuredTool(name='multiply_func', description='Multiply two integers.\n\nArgs:\n    a (int): The first integer.\n    b (int): The second integer.\n\nReturns:\n    int: The product of a and b.', args_schema=<class 'langchain_core.utils.pydantic.multiply_func'>, func=<function multiply_func at 0x000001D8E83FB240>)

In [25]:
class WeatherInput(BaseModel):
    city: str

In [26]:
def get_weather(city: str) -> str:
    """
    Get the weather for a given city.

    Args:
        city (str): The name of the city.

    Returns:
        str: A string describing the weather in the city.
    """
    return f"The weather in {city} is sunny."

In [ ]:
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="get_weather",
    description="Fetches real-time weather data for a city",
    args_schema=WeatherInput,
)

In [28]:
class WeatherInput(BaseModel):
    city: str = Field(..., description="City name")
    units: str = Field("metric", description="metric or imperial")

city: str = Field(..., description="City name")

- `city` must be a **string**.
- The `Field(...)` means this field is **required** — the `...` (ellipsis) signals Pydantic that this value **must** be provided.
- `description="City name"` is metadata useful for documentation (like in Swagger UI) to tell users what this field is.

units: str = Field("metric", description="metric or imperial")

- `units` is a **string**.
- This has a **default value** `"metric"` — so if the user doesn’t provide `units`, it defaults to `"metric"`.
- The description again is just for documentation, explaining this field should be `"metric"` or `"imperial"` (used for temperature units, e.g., Celsius or Fahrenheit).

`GetWeatherTool :`This class wraps a weather-fetching function into a "tool" with metadata and input validation so it can be used safely and clearly within an AI framework.

In [ ]:
from typing import ClassVar, Type

class GetWeatherTool(StructuredTool):
    name: ClassVar[str] = "get_weather"           
    description: ClassVar[str] = (
        "Fetches weather data for a city"
    )
    args_schema: ClassVar[Type[BaseModel]] = WeatherInput # validates input parameters the tool expects

    def _run(self, city: str, units: str = "metric") -> str: # when the tool is invoked, it takes city and optional units (default "metric") and calls the function get_weather to fetch the weather data
        return get_weather(city, units)